In [7]:
import os
import sys
import pandas as pd

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [3]:
import Connection as cn

In [4]:
spark = cn.spark_app_builder(cn.get_spark_master_ip(),
                             spark_app_name = "pyspark-taxi-forecasting")

INFO:ch:spark_master_ip: 172.18.0.2
DEBUG:ch:starting import SparkSession
INFO:ch:starting building spark app object: pyspark-taxi-forecasting
INFO:ch:builded spark app object: <pyspark.sql.session.SparkSession object at 0x7f116c626c90>


In [5]:
%%time
taxi = spark.read.load('/work/data/Taxi_Trips_-_2022.csv', 
                       format='csv', header='true', inferSchema='true')

CPU times: user 7.04 ms, sys: 9.9 ms, total: 16.9 ms
Wall time: 43 s


In [6]:
%%time
# Вывести схему данных
taxi.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

In [8]:
pd.set_option('display.max_colwidth', None)  # Показать полное содержимое ячеек столбцов

In [9]:
table_descr = pd.read_csv('/work/data/table_descr.csv',sep=';')

In [10]:
table_descr

,Column Name,Description,Type,Описание,Тип
0,Trip ID,A unique identifier for the trip.,Plain Text,Уникальный идентификатор поездки,Обычный текст
1,Taxi ID,A unique identifier for the taxi.,Plain Text,Уникальный идентификатор такси,Обычный текст
2,Trip Start Timestamp,When the trip started rounded to the nearest 15 minutes.,Date & Time,Время начала поездки округленное до ближайших 15 минут,Дата и время
3,Trip End Timestamp,When the trip ended rounded to the nearest 15 minutes.,Date & Time,Время окончания поездки округленное до ближайших 15 минут,Дата и время
4,Trip Seconds,Time of the trip in seconds.,Number,Продолжительность поездки в секундах,Число
5,Trip Miles,Distance of the trip in miles.,Number,Расстояние поездки в милях,Число
6,Pickup Census Tract,The Census Tract where the trip began. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где началась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
7,Dropoff Census Tract,The Census Tract where the trip ended. For privacy this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.,Plain Text,Код района по переписи населения где завершилась поездка. В целях конфиденциальности этот код не отображается для некоторых поездок. Это поле часто остается пустым для мест за пределами Чикаго.,Обычный текст
8,Pickup Community Area,The Community Area where the trip began. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где началась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число
9,Dropoff Community Area,The Community Area where the trip ended. This column will be blank for locations outside Chicago.,Number,Номер коммунальной области где завершилась поездка. Это поле остается пустым для мест за пределами Чикаго.,Число


In [11]:
pd.reset_option('display.max_colwidth')  # Сброс максимальной ширины столбцов к значению по умолчанию

In [12]:
%%time
describe_result = taxi.describe().toPandas()

CPU times: user 447 ms, sys: 104 ms, total: 551 ms
Wall time: 2min 54s


In [15]:
output_file_path = '/work/data/2022_describe.csv'
describe_result.to_csv(output_file_path, index=False) #transpose().

In [16]:
describe_result_22 = pd.read_csv('/work/data/2022_describe.csv',sep=',')

In [30]:
def get_column_types(dataframe):
    column_types = {}  # Словарь для хранения типов данных каждого столбца
    for column in dataframe.columns:
        # Сохраняем имя столбца и его тип данных в словарь
        column_types[column] = str(dataframe[column].dtype)
    return column_types

In [32]:
df = describe_result_22
# Пример использования функции
# Предположим, у вас есть DataFrame с именем df
# Используйте функцию следующим образом:
col_types = get_column_types(df)
print(col_types)

{'summary': 'object', 'Trip ID': 'object', 'Taxi ID': 'object', 'Trip Start Timestamp': 'object', 'Trip End Timestamp': 'object', 'Trip Seconds': 'float64', 'Trip Miles': 'float64', 'Pickup Census Tract': 'float64', 'Dropoff Census Tract': 'float64', 'Pickup Community Area': 'float64', 'Dropoff Community Area': 'float64', 'Fare': 'float64', 'Tips': 'float64', 'Tolls': 'float64', 'Extras': 'float64', 'Trip Total': 'float64', 'Payment Type': 'object', 'Company': 'object', 'Pickup Centroid Latitude': 'float64', 'Pickup Centroid Longitude': 'float64', 'Pickup Centroid Location': 'object', 'Dropoff Centroid Latitude': 'float64', 'Dropoff Centroid Longitude': 'float64', 'Dropoff Centroid  Location': 'object'}


In [27]:
pd.options.display.float_format = '{:,.2f}'.format  # Ограничиваем количество знаков после запятой до 2
describe_result_22_formatted = describe_result_22.copy()  # Создаем копию, чтобы избежать изменения оригинального датафрейма

# Разделить разряды в числах, начиная с 0-й строки и столбца
for col in describe_result_22_formatted.columns:
    if describe_result_22_formatted[col].dtype == 'float64' or describe_result_22_formatted[col].dtype == 'int64':
        describe_result_22_formatted[col] = describe_result_22_formatted[col].apply(lambda x: '{:,.0f}'.format(x).replace(',', "'") if pd.notnull(x) else x)

print(describe_result_22_formatted
      # .transpose()
     )

  summary                                   Trip ID  \
0   count                                   6382425   
1    mean                                       NaN   
2  stddev                                       NaN   
3     min  000000bb18f0563c13ad977fc05b901474cd3941   
4     max  ffffff1aae5322736637e16dd2faecb5dfebe81a   

                                             Taxi ID    Trip Start Timestamp  \
0                                            6382425                 6382425   
1                                                NaN                     NaN   
2                                                NaN                     NaN   
3  0041f8f0c91881c1e1913f2548522495fe3c4c719aa67f...  01/01/2022 01:00:00 AM   
4  fff84aa08ac78890c6e7da64b817cbd9aad6a124104e09...  12/31/2022 12:45:00 PM   

       Trip End Timestamp Trip Seconds Trip Miles Pickup Census Tract  \
0                 6382213    6'380'960  6'382'369           2'623'831   
1                     NaN        1'198    